In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.base import clone
import joblib
import gc

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Set up white-grid plot style
sns.set_style("whitegrid")

# Optional: Set a color palette (you can choose a different one if you prefer)
sns.set_palette("deep")

CWD = os.getcwd()
print(f'CWD: {CWD}')

ROOT = os.path.abspath(os.path.dirname(os.path.dirname(CWD)))
sys.path.append(ROOT)

from utils.visualization import boxplot_by_category, stacked_barplot_by_category
from utils.tuning import instantiate_model
from utils.utils import Dataloader

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CWD: /data_analysis/Insurance/src


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
PROJ = os.path.dirname(CWD)
TRAIN_PATH = [os.path.join(PROJ, 'data', x)  for x in ['train.csv', 'train_orig.csv']]
TEST_PATH = os.path.join(PROJ, 'data', 'test.csv')

target = 'Response'
num_cols = ['Age', 'Annual_Premium', 'Vintage']
cat_cols = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Vehicle_Age']

# train = pd.concat([pd.read_csv(PATH, index_col='id') for PATH in TRAIN_PATH]).reset_index(drop=True)
train = pd.read_csv( TRAIN_PATH[0], index_col = 'id')
test = pd.read_csv(TEST_PATH, index_col = 'id')

for col in cat_cols:
    train[col], test[col] = train[col].astype('category'), test[col].astype('category')

# train = train.sample(1000)

In [3]:
N_FOLDS = 5
MODEL = 'XGBoost'
def objective(trial):
    score = 0
    # Call the function to instantiate the model
    model = instantiate_model(trial, num_cols, cat_cols, datetime_columns=None, string_columns=None, models=[MODEL])

    X = train.drop(target, axis=1)
    y = train[target]

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

    f1_scores = []

    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Fit the model
        m = clone(model)
        m.fit(X_train, y_train)

        # Predict on validation set
        y_pred = m.predict_proba(X_val)

        # Calculate F1 score
        score += roc_auc_score(y_val, y_pred[:, 1])/N_FOLDS

        del m
        gc.collect()

    return score

## Hyper parameter optimization

In [4]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')

# Seed using parameters from public notebook
if MODEL == 'XGBoost':
    study.enqueue_trial(
        {
            'xgb_alpha': 1.302348865795227e-06,
            'xgb_max_depth': 15, 
            'xgb_learning_rate': 0.061800451723613786, 
            'xgb_subsample': 0.7098803046786328, 
            'xgb_colsample_bytree': 0.2590672912533101, 
            'xgb_min_child_weight': 10, 
            'xgb_gamma': 0.8399887056014855, 
            'xgb_reg_alpha': 0.0016943548302122801, 
            'xgb_max_bin': 71284,
            'xgb_n_estimators': 1200,

        }
    )
study.optimize(objective, n_trials=100, show_progress_bar=True, gc_after_trial=True)  # You can adjust the number of trials

[I 2024-07-23 01:31:32,147] A new study created in memory with name: no-name-8478d28e-411f-4360-940a-62da9308b90e
Best trial: 0. Best value: 0.842797:   1%|          | 1/100 [05:45<9:30:16, 345.62s/it]

[I 2024-07-23 01:37:17,696] Trial 0 finished with value: 0.8427965426416549 and parameters: {'numerical_strategy': 'median', 'with_centering': False, 'with_scaling': False, 'categorical_strategy': 'most_frequent', 'smoothing': 37.95696709232721, 'min_samples_leaf': 26, 'clf': 'XGBoost', 'xgb_alpha': 1.302348865795227e-06, 'xgb_max_depth': 15, 'xgb_learning_rate': 0.061800451723613786, 'xgb_n_estimators': 1200, 'xgb_min_child_weight': 10, 'xgb_subsample': 0.7098803046786328, 'xgb_colsample_bytree': 0.2590672912533101, 'xgb_reg_alpha': 0.0016943548302122801, 'xgb_reg_lambda': 0.14372827107324893, 'percentile': 47}. Best is trial 0 with value: 0.8427965426416549.


Best trial: 1. Best value: 0.86257:   2%|▏         | 2/100 [17:35<15:14:02, 559.61s/it]

[I 2024-07-23 01:49:07,114] Trial 1 finished with value: 0.8625701498569582 and parameters: {'numerical_strategy': 'median', 'with_centering': True, 'with_scaling': False, 'categorical_strategy': 'most_frequent', 'smoothing': 66.77746736625397, 'min_samples_leaf': 33, 'clf': 'XGBoost', 'xgb_alpha': 0.0639877641744776, 'xgb_max_depth': 18, 'xgb_learning_rate': 0.17289588872212988, 'xgb_n_estimators': 971, 'xgb_min_child_weight': 3, 'xgb_subsample': 0.9919503912765959, 'xgb_colsample_bytree': 0.9550823556271376, 'xgb_reg_alpha': 4.232175400270493, 'xgb_reg_lambda': 8.70492781399606, 'percentile': 70}. Best is trial 1 with value: 0.8625701498569582.


Best trial: 1. Best value: 0.86257:   3%|▎         | 3/100 [21:24<11:01:05, 408.92s/it]

...

[I 2024-07-23 04:29:18,874] Trial 26 finished with value: 0.8643225283067304 and parameters: {'numerical_strategy': 'mean', 'with_centering': True, 'with_scaling': True, 'categorical_strategy': 'constant', 'smoothing': 2.1604150234150783, 'min_samples_leaf': 37, 'clf': 'XGBoost', 'xgb_alpha': 0.00961226329133488, 'xgb_max_depth': 6, 'xgb_learning_rate': 0.21419187710574633, 'xgb_n_estimators': 883, 'xgb_min_child_weight': 8, 'xgb_subsample': 0.6578352539506586, 'xgb_colsample_bytree': 0.5439909128991353, 'xgb_reg_alpha': 4.553532506094161, 'xgb_reg_lambda': 3.876705942436258, 'percentile': 65}. Best is trial 24 with value: 0.8803920313223899.


Best trial: 24. Best value: 0.880392:  28%|██▊       | 28/100 [3:03:14<6:44:30, 337.09s/it]

[I 2024-07-23 04:34:46,653] Trial 27 finished with value: 0.8792472571257642 and parameters: {'numerical_strategy': 'most_frequent', 'with_centering': False, 'with_scaling': False, 'categorical_strategy': 'most_frequent', 'smoothing': 13.282540209473778, 'min_samples_leaf': 31, 'clf': 'XGBoost', 'xgb_alpha': 0.00906052055708545, 'xgb_max_depth': 4, 'xgb_learning_rate': 0.15686279627651012, 'xgb_n_estimators': 933, 'xgb_min_child_weight': 8, 'xgb_subsample': 0.6057967853100638, 'xgb_colsample_bytree': 0.5530508357276738, 'xgb_reg_alpha': 2.122959611179023, 'xgb_reg_lambda': 9.999843699109977, 'percentile': 90}. Best is trial 24 with value: 0.8803920313223899.


Best trial: 24. Best value: 0.880392:  29%|██▉       | 29/100 [3:06:49<5:55:34, 300.49s/it]

[I 2024-07-23 04:38:21,762] Trial 28 finished with value: 0.8363991952638603 and parameters: {'numerical_strategy': 'constant', 'with_centering': False, 'with_scaling': False, 'categorical_strategy': 'most_frequent', 'smoothing': 8.721636109614288, 'min_samples_leaf': 40, 'clf': 'XGBoost', 'xgb_alpha': 0.00509724862973272, 'xgb_max_depth': 7, 'xgb_learning_rate': 0.2171801955859604, 'xgb_n_estimators': 207, 'xgb_min_child_weight': 9, 'xgb_subsample': 0.661922085112359, 'xgb_colsample_bytree': 0.40343126034823695, 'xgb_reg_alpha': 2.0080005586030296, 'xgb_reg_lambda': 5.863081057791678, 'percentile': 39}. Best is trial 24 with value: 0.8803920313223899.


In [ ]:
# Print the best parameters and best value
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

final_score = f'{trial.value*100:.3f}'.replace('.', '')
FILE_NAME = f'{MODEL}_{final_score}'

# You can now use the best parameters to create your final model
best_model = instantiate_model(trial, num_cols, cat_cols, datetime_columns=None, string_columns=None, models=[MODEL])

# Re-fit Final model
X = train.drop(target, axis=1)
y = train[target]
best_model.fit(X, y)
y_preds_proba = best_model.predict_proba(test)

model_filename = os.path.join(PROJ, 'out', f'{FILE_NAME}.joblib')
joblib.dump(trial, model_filename)

## Create Submission file

First file uses the prediction of the model.
Second file uses the information that the synthetic data often contains the same rows as the original training data that generated it, but with the responses flipped. Using that information, we override the values in the first file using information from the original data set.

In [ ]:
# Create submission file
SUB_PATH = os.path.join(PROJ, 'data', 'sample_submission.csv')
OUT_PATH = os.path.join(PROJ, 'out', f'{FILE_NAME}.csv')
sub = pd.read_csv(SUB_PATH)
sub['Response'] = y_preds_proba[:, 1]
sub.to_csv(OUT_PATH, index=False)

sub.shape


In [ ]:
#  Find all matching data from the original train data set on the synthetic test dataset
orig_train = pd.read_csv( TRAIN_PATH[1], index_col = 'id')
match_columns = sorted(set(test.columns) - set(['id']))
test_merged = pd.merge(test.reset_index(), orig_train[match_columns + ['Response']], on=match_columns, how='inner').drop_duplicates().set_index('id')
test_merged['Response'] = 1 - test_merged['Response']


# Override the model predictions
sub.set_index('id', inplace=True)
matching_ids = sub.index.intersection(test_merged.index)
sub.loc[matching_ids, 'Response'] = test_merged.loc[matching_ids, 'Response']
sub.reset_index(inplace=True)

OUT_PATH = os.path.join(PROJ, 'out', f'{FILE_NAME}_override.csv')
sub.to_csv(OUT_PATH, index=False)
sub.shape